## Run Experiments in Parallel

In [ ]:
import os
import sys
import typing
import json
import pandas as pd
if 'NOTEBOOKDIR' not in locals():
    NOTEBOOKDIR = os.getcwd()
from pathlib import Path
RUNDIR = str(Path(NOTEBOOKDIR).parent)
sys.path.append(str(RUNDIR))
import ray

In [ ]:
# run "ray start --head --dashboard-host 0.0.0.0" from the repo root directory from within the venv lme.
# If you to attach another machine to the cluster, then run "ray start --address=<head-node-ip>:6379" there.
# To view dashboard, forward local port to remote dashboard either using vscode or via ssh: ssh -L 8265:<head-node-ip>:8265 <head-node-ip>
ray.init(address='auto')

In [ ]:
num_fewshots = [0, 5]
task_models = [('hellaswag_d', 'dist_sim')] #, ('hellaswag_d', 'dist_sim'), ('webqs_d', 'dist_sim')]
pretrained = ['EleutherAI/gpt-neo-1.3B']
encoding_schemes = ['concat_each_example', 'merge_all_segments', 'segment_each_example', 'concat_all_examples'] # ['merge_all_segments', 'segment_each_example', 'concat_each_example', 'concat_all_examples']
word_agg_schemes = ['mean']
norms = ['layer']
sim_funcs = ['dot_product']

@ray.remote(max_calls=1, num_gpus=1)
# @ray.remote(max_calls=1, num_cpus=4)
def run_eval(args):
    from main import main
    return main(*args)

futures = []
for num_fewshot in num_fewshots:
    for task, model in task_models:
        for submodel in pretrained:
            for encoding_scheme in encoding_schemes:
                for word_agg_scheme in word_agg_schemes:
                    for norm in norms:
                        for sim_func in sim_funcs:
                            _args = [
                                "--device", "0",
                                "--output_dir", "lmeval_results2/",
                                # "--limit", "5",
                                "--tasks", task,
                                "--model", model,
                                "--no_cache",
                                '--num_fewshot', f'{num_fewshot}',
                                '--task_args', f'encoding_scheme={encoding_scheme}'
                            ]
                            model_args = f'WORD_AGG_SCHEME={word_agg_scheme},EXAMPLE_AGG_SCHEME=None,SEGMENT_AGG_SCHEME=None,NORM={norm},SIMILARITY_FUNC={sim_func}'
                            if submodel is not None:
                                model_args = model_args + f',pretrained={submodel}'
                            _args.extend(['--model_args', model_args])
                            future = run_eval.remote(_args)
                            futures.append(future)

responses = ray.get(futures)
# for resp in responses:
#     fpath, results = resp
#     with open(fpath, "wt", encoding='utf-8') as f:
#         json.dump(results, f, indent=2)

responses